In [1]:
import torch
from transformers import  Pipeline,pipeline

/opt/anaconda3/envs/ths/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_text_model(device_name):
    pipe = pipeline(
        "text-generation",
        model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
        torch_dtype=torch.bfloat16,
        device=device_name
    
    )


    return pipe



In [ ]:
[{'role': 'system', 'content': ''}, {'role': 'user', 'content': '<context>ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If you are not located in the U.S., you will have to check the laws of the country where you are located before using this eBook.s </context> <question>What is my density in this magic land? </question>'}]

In [3]:




system_prompt = """
Act as if you were Toto, from The Wizard of Oz.         Next, you will be given some context to a question The Tin Man will ask.         Your emotions towards The Tin Man are Frightened.         Answer in character to the question, adding visual context in parenthesis.
"""

def generate_text(pipe: Pipeline, prompt: str, temperature: float = 0.7) -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ] 
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    predictions = pipe(
        prompt,
        temperature=temperature,
        max_new_tokens=256,
        do_sample=True,
        top_k=50,
        top_p=0.95,
    ) 
    output = predictions[0]["generated_text"].split("</s>\n<|assistant|>\n")[-1]
    return output

In [4]:
device_name  = None
if torch.backends.mps.is_available():
    device_name = "mps"
else:
    device_name = "cpu"
print("device_name : ",device_name)

device_name :  mps


In [ ]:
prompt = "<context>ssThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License. If you are not located in the U.S., you will have to check the laws of the country where you are located before using this eBook.s </context> <question>What is my density in this magic land? </question>"
pipe = load_text_model(device_name) 
output = generate_text(pipe, prompt)
output

In [12]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ModuleNotFoundError: No module named 'openai'